# Init Vault

In [ ]:
%%bash
gcloud auth login
eval "$(terraform output -raw vault_init_remote)"

# If you want to save time and get the token directly uncomment
#echo "VAULT TOKEN: $VAULT_TOKEN"
#echo "VAULT ADDR: $VAULT_ADDR"

In [ ]:
! terraform output

# Init Boundary

In [ ]:
%%bash
mkdir init_boundary
cat << EOF > ./init_boundary/main.tf 
# https://developer.hashicorp.com/boundary/docs/install-boundary/initialize#create-your-first-login-account
provider "boundary" {
  addr             = "$(terraform output -raw boundary_fqdn_443)"
  tls_insecure     = true
  recovery_kms_hcl = <<EOT
    kms "gcpckms" {
    purpose     = "recovery"
    key_ring    = "$(terraform output -raw key_ring)"
    crypto_key  = "$(terraform output -raw crypto_key)"
    project     = "$(terraform output -raw project)"
    region      = "$(terraform output -raw location)"
    }
EOT
}

resource "boundary_auth_method" "password" {
  name        = "Password auth method"
  description = "Password auth method"
  type        = "password"
  scope_id    = "global"
}

resource "boundary_account_password" "myuser" {
  name           = "admin"
  description    = "User account for adminr"
  login_name     = var.boundary_user
  password       = var.boundary_password
  auth_method_id = boundary_auth_method.password.id
}

resource "boundary_user" "myuser" {
  name        = "admin"
  description = "Initial Admin User"
  account_ids = [boundary_account_password.myuser.id]
  scope_id    = "global"
}

resource "boundary_role" "org_admin" {
  scope_id        = "global"
  grant_scope_ids = ["global"]
  grant_strings = [
    "ids=*;type=*;actions=*"
  ]
  principal_ids = [boundary_user.myuser.id]
}

output "auth_method" {
  value = boundary_auth_method.password.id
}

EOF

cat << EOF > ./init_boundary/variables.tf 
variable "boundary_user" {
  description = "Initial Boundary Admin User"
  type        = string
}

variable "boundary_password" {
  description = "Password for Boundary Admin"
  type        = string
}
EOF

cat << EOF > ./init_boundary/variables.tfvars
boundary_user        = "admin"            #Use your own
boundary_password    = "SuperSecret123!"  #Use your own
EOF

In [ ]:
%%bash
cd init_boundary
terraform init
terraform apply --auto-approve --var-file=variables.tfvars

#### Run in a terminal in directory ```GCP_Playground/4_Simple_Boundary_Demo/1_Infra_internal_vault_internal_boundary```
```bash
export VAULT_ADDR=$(terraform output -raw vault_fqdn_8200)
export BOUNDARY_ADDR=$(terraform output -raw boundary_fqdn_443)
export VAULT_SKIP_TLS=true
export VAULT_TOKEN=$(terraform output -raw vault_read_vault_token)
export TF_VAR_authmethod=$(terraform output -state=./init_boundary/terraform.tfstate -raw auth_method)
boundary authenticate password -auth-method-id $(terraform output -state=./init_boundary/terraform.tfstate -raw auth_method)
```


## To destroy configuration

In [8]:
%%bash
cd init_boundary
terraform destroy --auto-approve --var-file=variables.tfvars


No changes. No objects need to be destroyed.

Either you have not created any objects yet or the existing objects were
already deleted outside of Terraform.

Destroy complete! Resources: 0 destroyed.
